In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os
import requests_cache

from dataclasses import dataclass
from pydantic import BaseModel, Field
from pydantic_ai import RunContext
from pydantic_ai import Agent, ToolOutput
from pydantic_ai.mcp import MCPServerStdio
from pydantic_ai.messages import ModelRequest, UserPromptPart
from pydantic_ai.models.openai import ModelRequestParameters, OpenAIModel, OpenAIModelSettings
from pydantic_ai.providers.openai import OpenAIProvider
from typing import List


model = OpenAIModel(
    'Qwen/Qwen3-8B-FP8',
    provider=OpenAIProvider(
        base_url='http://localhost:8000/v1', api_key=os.getenv("VLLM_API_KEY")
    ),
)

model.model_name

In [ ]:
class Answer(BaseModel):  
    answer: str | int | float
    

web_access_server = MCPServerStdio(  
    'uv',
    args=[
        'run',
        'mcp',
        'run',
        '-t',
        'stdio',
        'web_access_mcp_server.py',
    ],
    env={
        "SERPER_API_TOKEN": os.getenv("SERPER_API_TOKEN"),
        "LLM_API_KEY": os.getenv("VLLM_API_KEY")
    },
)

python_server = MCPServerStdio(  
    'deno',
    args=[
        'run',
        '-N',
        '-R=node_modules',
        '-W=node_modules',
        '--node-modules-dir=auto',
        'jsr:@pydantic/mcp-run-python',
        'stdio',
    ],
)


instructions = (
    "You are a helpful assistant."
    "\n\n"
    "Don't do math calculations yourself!!! "
    "Always use a Python tool to do math calculations."
    "\n\n"
    "You can search the internet information. "
    "First, search for links or papers that might be relevant to your task. "
    "Then read a link or paper that fits the query. "
    "However, don't read anything that is homework or a dataset. "
    "Stop reading once you have found the requested information. "
    "If the links don't seem promising, search again with a different query. "
    "You do not have the ability to watch videos, so if you are asked to find information on a video platform, read the video description for clues."
)
      
agent = Agent(
    model,
    model_settings=OpenAIModelSettings(parallel_tool_calls=False, temperature=0),
    output_type=ToolOutput(Answer, name="final_answer", description="Call this tool once you've completed your task."),
    instructions=instructions,
    toolsets=[python_server, web_access_server],
)

In [ ]:
nodes = []
# Begin an AgentRun, which is an async-iterable over the nodes of the agent's graph
async with agent.iter("Where can I get good food in San Francisco? Please read the reviews to be sure.") as agent_run:
    async for node in agent_run:
        # Each node represents a step in the agent's execution
        nodes.append(node)
        print(node, end="\n\n")

In [ ]:
print(nodes[-1].data.output.answer)